# QA Micropal_headers_PBDB_Taxonomy_notes_taxa_list

Check if the taxa that were aproved by the researchers from the google sheet matches the taxa from the LIMS taxa_list.csv.

In [1]:
import pandas as pd
import numpy as np

In [2]:
date = '2021-05-13'
input_file = f'raw_data/taxa/Micropal_headers_PBDB_Taxonomy_notes_taxa_list_{date}.csv'

taxon_groups = [
    "nannofossils",
    "silicoflagellates",
    "ostracods",
    "ebridians",
    "chrysophyte_cysts",
    "bolboformids",
    "diatoms",
    "planktic_forams",
    "radiolarians",
    "dinoflagellates",
    "palynology",
]

In [3]:
def add_normalized_name_column(df):
    fields = [
        "genus modifier",
        "genus name",
        "subgenera modifier",
        "subgenera name",
        "species modifier",
        "species name",
        "subspecies modifier",
        "subspecies name",
    ]

    # concatenate taxa fields into a string
    df["normalized_name"] = df["Any taxon above genus"].str.cat(
        df[fields], sep=" ", na_rep=""
    )

    # add "(descriptor)" if it exists
    descriptor = np.where(
        df["non-taxa descriptor"].notnull(), "(" + df["non-taxa descriptor"] + ")", ""
    )
    df["normalized_name"] = df["normalized_name"] + descriptor

    # get rid of extra spaces
    df["normalized_name"] = df["normalized_name"].str.strip()
    df["normalized_name"] = df["normalized_name"].replace(
        to_replace="  +", value=" ", regex=True
    )

    return df

## compare LIMS taxa with approved taxa

In [4]:
all_taxa_file = 'cleaned_data/taxa/taxa_list.csv'

In [5]:
normalized_df = pd.read_csv(input_file, skiprows = 9)
normalized_df.shape

(4754, 21)

In [6]:
normalized_df.head(2)

,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,notes,Any taxon above genus,genus modifier,genus name,...,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,comments,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank
0,NaN,RESOLVED NAMES BY TAXONOMIC GROUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,benthic_forams,Pyrite,Pyrite,NaN,NaN,"not a taxa; Going into Macrostrat, not PBDB",Summer 2020: deal with later; should go in as ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
taxa_df = pd.read_csv(all_taxa_file)
taxa_df.shape

(4738, 4)

In [8]:
taxa_df.head(2)

,verbatim_name,name,taxon_group,genera
0,"""Globigerina"" angulisuturalis _T","""Globigerina"" angulisuturalis _T",planktic_forams,NaN
1,"""Globigerina"" angulisuturalis _T_","""Globigerina"" angulisuturalis _T_",planktic_forams,NaN


In [9]:
normalized_names = set(normalized_df['verbatim_name'])
taxa_names = set(taxa_df['verbatim_name'])

get taxa in the normalized taxa file that isn't in LIMS taxa file

In [10]:
len(normalized_names - taxa_names)

2

In [11]:
normalized_names - taxa_names

{'RESOLVED NAMES BY TAXONOMIC GROUP', nan}

get taxa in the  LIMS taxa file  that isn't in normalized taxa file

In [12]:
len(taxa_names - normalized_names)

1

In [13]:
taxa_names - normalized_names

{'Sponge spicules'}

## check for duplicate taxa


In [14]:
temp_df = normalized_df.copy()
temp_df = temp_df.drop(list(range(29)))
temp_df.head(2)

,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,notes,Any taxon above genus,genus modifier,genus name,...,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,comments,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank
29,planktic_forams,Globigerina falconensis,Globigerina falconensis,NaN,NaN,NaN,"in PBDB, so it will be classified correctly; t...",NaN,NaN,Globigerina,...,NaN,NaN,falconensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,planktic_forams,Globigerina rubescens,Globigerina rubescens,NaN,NaN,NaN,"in PBDB, so it will be classified correctly; t...",NaN,NaN,Globigerina,...,NaN,NaN,rubescens,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
temp_df.shape

(4725, 21)

In [16]:
filtered_taxa = temp_df[temp_df['taxon_group'].isin(taxon_groups)].copy()
filtered_taxa.shape

(3296, 21)

In [17]:
add_normalized_name_column(filtered_taxa)
filtered_taxa.head(2)

,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,notes,Any taxon above genus,genus modifier,genus name,...,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,comments,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,normalized_name
29,planktic_forams,Globigerina falconensis,Globigerina falconensis,NaN,NaN,NaN,"in PBDB, so it will be classified correctly; t...",NaN,NaN,Globigerina,...,NaN,falconensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Globigerina falconensis
30,planktic_forams,Globigerina rubescens,Globigerina rubescens,NaN,NaN,NaN,"in PBDB, so it will be classified correctly; t...",NaN,NaN,Globigerina,...,NaN,rubescens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Globigerina rubescens


In [18]:
group_taxa = {}

for index, row in filtered_taxa.iterrows():
    if row['normalized_name'] not in group_taxa:
        group_taxa[row['normalized_name']] = set()
    
    group_taxa[row['normalized_name']].add(row['taxon_group'])  


In [19]:
taxa_tmp = []
taxon_groups_tmp = []

for key, value in group_taxa.items():
    if len(value) > 1 and key:
        taxa_tmp.append(key)
        taxon_groups_tmp.append(', '.join(value))

In [20]:
dict = {
    "taxon": taxa_tmp,
    "taxon_group": taxon_groups_tmp
}
df = pd.DataFrame(dict)
df.tail()

,taxon,taxon_group
3,Foraminifera indet.,"dinoflagellates, planktic_forams, palynology"
4,Brigantedinium spp.,"dinoflagellates, palynology"
5,Lejeunecysta sp.,"dinoflagellates, palynology"
6,Selenopemphix nephroides,"dinoflagellates, palynology"
7,Dinoflagellata indet.,"dinoflagellates, nannofossils, palynology"


In [21]:
df.to_csv('./tmp/taxa_count.csv', index=False)
df.shape

(8, 2)